<a href="https://colab.research.google.com/github/Billyjr2003/deep-learning/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torch.nn.functional as fc
from torch.utils.data import dataloader

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [55]:
train_data = dataset.MNIST(root='dataset/', train= True, transform= transforms.ToTensor(), download= True)
test_data = dataset.MNIST(root='dataset/', train= False, transform= transforms.ToTensor(), download= True)

In [56]:
# Hyperparameters
input_size = 28
num_seq = 28
hidden_size = 256
num_layers = 2
num_classes = 10
batch_size = 64
l_r = 0.001
num_epochs = 5

In [57]:
train_data_loader = dataloader.DataLoader(dataset= train_data, batch_size= batch_size, shuffle= True)
test_data_loader = dataloader.DataLoader(dataset= test_data, batch_size= batch_size, shuffle= False)

In [58]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first= True, bidirectional= True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    h_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    out, _ = self.lstm(x, (h_0, c_0))
    out = self.fc(out[:, -1, :])
    return out

In [59]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [60]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= l_r)

In [61]:
def accuarcy(loader, model_cls, device):
  num_correct = 0
  num_samples = 0
  model_cls.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)
      scores = model_cls(x)
      predections = torch.argmax(scores, dim = 1)
      num_correct += (predections == y).sum()
      num_samples += predections.size(0)
    print(f'Got {num_correct / num_samples} with accuracy {(float(num_correct)/ float(num_samples))* 100} %')
  model_cls.train()

**LSTM Model**

In [63]:
for epoch in range(num_epochs):
  for batch , (data, target) in enumerate(train_data_loader):
    data = data.to(device).squeeze(1)
    target = target.to(device)
    pred = model(data)

    loss_fn = loss(pred, target)

    optimizer.zero_grad()
    loss_fn.backward()
    optimizer.step()

accuarcy(loader=train_data_loader, model_cls= model, device= device)
accuarcy(loader=test_data_loader, model_cls= model, device= device)

Got 0.9879000186920166 with accuracy 98.79 %
Got 0.9839999675750732 with accuracy 98.4 %
